In [2]:
import random

def generate_graph_data(num_timestamps=3, num_artists=10, edge_style="stable"):
    """
    Generate graph data for multiple timestamps with a specified number of artists
    and an edge pattern style.

    :param num_timestamps: Number of discrete time steps to generate (e.g., 3).
    :param num_artists: Number of artists (nodes) in each timestamp (e.g., 10).
    :param edge_style: A descriptor for how edges change over time. Could be one of:
                      ["stable", "dynamic", "emerging_clusters", "random_disconnected", etc.]
    :return: A dictionary keyed by timestamps (e.g., 't_0','t_1',...) each containing:
             {
                 'nodes': [ { 'id', 'name', 'num_exhibitions' } ... ],
                 'links': [ { 'source', 'target', 'weight', 'influence' } ... ],
                 'num_exhibitions': <some_aggregate_value>
             }
    """

    # You can parametrize how you label timestamps if you prefer "t_1905", "t_1906", etc.
    # For simplicity, we'll use "t_0", "t_1", ...
    
    graph_data = {}

    # ------------------------------------------------
    # STEP 1: Pre-generate all artists.
    # ------------------------------------------------
    # We will have the same set of artists in each timestamp with stable IDs.
    # (Alternatively, you could create new artists over time if needed.)
    
    # Generate node info
    artists = []
    for artist_id in range(num_artists):
        # Randomize or pattern the number of exhibitions
        num_exhibitions = random.randint(1, 7)
        artists.append({
            "id": artist_id,
            "name": f"Artist {artist_id}",
            "num_exhibitions": num_exhibitions
        })

    # ------------------------------------------------
    # STEP 2: Define various "edge styles".
    # ------------------------------------------------
    
    def generate_stable_edges():
        """
        In a 'stable' style, we pick a certain set of edges once
        and reuse them in each timestamp (with minimal variation).
        """
        possible_pairs = []
        for i in range(num_artists):
            for j in range(i+1, num_artists):
                possible_pairs.append((i, j))
                
        # We want a stable set of edges, so pick a subset from possible pairs
        # e.g., 1/3 of all possible
        num_edges = max(1, len(possible_pairs) // 3)
        chosen_pairs = random.sample(possible_pairs, num_edges)
        
        # Return a function that, given a timestamp index, returns the same edges
        def edges_for_t(t):
            edges = []
            for (src, tgt) in chosen_pairs:
                edges.append({
                    "source": src,
                    "target": tgt,
                    "weight": 1,
                    "influence": 0.0
                })
            return edges
        return edges_for_t

    def generate_dynamic_edges():
        """
        In a 'dynamic' style, each timestamp has a new random set of edges.
        """
        def edges_for_t(t):
            possible_pairs = []
            for i in range(num_artists):
                for j in range(i+1, num_artists):
                    possible_pairs.append((i, j))
            
            # Random fraction of edges each time
            num_edges = random.randint(num_artists, len(possible_pairs)//2)
            chosen_pairs = random.sample(possible_pairs, num_edges)
            
            edges = []
            for (src, tgt) in chosen_pairs:
                edges.append({
                    "source": src,
                    "target": tgt,
                    "weight": 1,
                    "influence": 0.0
                })
            return edges
        return edges_for_t

    def generate_emerging_clusters_edges():
        """
        In an 'emerging_clusters' style, we can start with two disjoint or lightly connected
        clusters that gradually merge or connect more strongly over time.
        """
        # Predefine two clusters
        cluster_size = num_artists // 2
        cluster_A = list(range(cluster_size))
        cluster_B = list(range(cluster_size, num_artists))
        
        # The idea: at t=0, mostly edges within each cluster; at t=last, more edges crossing.
        
        def edges_for_t(t):
            # fraction that indicates how much "cross-cluster" connectivity we add
            fraction_cross = t / max(1, (num_timestamps - 1))
            
            edges = []
            # Always connect within the clusters fairly densely
            for i in cluster_A:
                for j in cluster_A:
                    if i < j and random.random() < 0.6:  # 60% chance within cluster
                        edges.append({
                            "source": i,
                            "target": j,
                            "weight": 1,
                            "influence": 0.0
                        })
            for i in cluster_B:
                for j in cluster_B:
                    if i < j and random.random() < 0.6:
                        edges.append({
                            "source": i,
                            "target": j,
                            "weight": 1,
                            "influence": 0.0
                        })
            
            # Cross-cluster edges in proportion to fraction_cross
            num_cross_edges = int((len(cluster_A) * len(cluster_B)) * fraction_cross * 0.2)
            
            all_cross_pairs = []
            for a in cluster_A:
                for b in cluster_B:
                    all_cross_pairs.append((a, b))
            
            if num_cross_edges > 0:
                chosen_cross = random.sample(all_cross_pairs, min(num_cross_edges, len(all_cross_pairs)))
                for (src, tgt) in chosen_cross:
                    edges.append({
                        "source": src,
                        "target": tgt,
                        "weight": 1,
                        "influence": 0.0
                    })
            
            return edges
        
        return edges_for_t

    # ------------------------------------------------
    # STEP 3: Choose an edge generator based on edge_style
    # ------------------------------------------------
    if edge_style == "stable":
        edge_generator = generate_stable_edges()
    elif edge_style == "dynamic":
        edge_generator = generate_dynamic_edges()
    elif edge_style == "emerging_clusters":
        edge_generator = generate_emerging_clusters_edges()
    else:
        # Default: treat it like dynamic
        edge_generator = generate_dynamic_edges()

    # ------------------------------------------------
    # STEP 4: Build up each timestamp's graph data
    # ------------------------------------------------
    
    for t in range(num_timestamps):
        timestamp_label = f"t_{t}"  # e.g., "t_0", "t_1", ...
        
        # Copy the node list (so each timestamp has its own list instance)
        nodes_this_t = []
        for node in artists:
            # Optionally you could randomize 'num_exhibitions' a bit more per timestamp
            # but here we’ll keep them stable for demonstration
            nodes_this_t.append({
                "id": node["id"],
                "name": node["name"],
                "num_exhibitions": node["num_exhibitions"]
            })
        
        # Generate edges using the chosen style
        edges_this_t = edge_generator(t)
        
        # Example: we set "num_exhibitions" at the snapshot level to the sum or max
        total_exhibitions = sum(node["num_exhibitions"] for node in nodes_this_t)
        
        graph_data[timestamp_label] = {
            "nodes": nodes_this_t,
            "links": edges_this_t,
            "num_exhibitions": total_exhibitions
        }

    return graph_data


# ------------------------------
# Example usage and printing:
# ------------------------------
if __name__ == "__main__":
    # Generate 3 timestamps, with 10 artists, using an "emerging_clusters" style
    data_example = generate_graph_data(
        num_timestamps=30, 
        num_artists=1, 
        edge_style="emerging_clusters"
    )
    
    import json
    print(json.dumps(data_example, indent=4))



{
    "t_0": {
        "nodes": [
            {
                "id": 0,
                "name": "Artist 0",
                "num_exhibitions": 5
            }
        ],
        "links": [],
        "num_exhibitions": 5
    },
    "t_1": {
        "nodes": [
            {
                "id": 0,
                "name": "Artist 0",
                "num_exhibitions": 5
            }
        ],
        "links": [],
        "num_exhibitions": 5
    },
    "t_2": {
        "nodes": [
            {
                "id": 0,
                "name": "Artist 0",
                "num_exhibitions": 5
            }
        ],
        "links": [],
        "num_exhibitions": 5
    },
    "t_3": {
        "nodes": [
            {
                "id": 0,
                "name": "Artist 0",
                "num_exhibitions": 5
            }
        ],
        "links": [],
        "num_exhibitions": 5
    },
    "t_4": {
        "nodes": [
            {
                "id": 0,
                "name":